In [119]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn.functional import softmax
import transformers
from transformers import AutoModel, BertTokenizerFast

# ГПУ для більших швидких обчислень
device = torch.device("cuda")

In [120]:
data = pd.read_csv("../input/sarcasm-detection/sarcasm_detection.csv")
data = data[['headline', 'is_sarcastic']]
data.head()

,headline,is_sarcastic
0,thirtysomething scientists unveil doomsday clo...,1
1,dem rep. totally nails why congress is falling...,0
2,eat your veggies: 9 deliciously different recipes,0
3,inclement weather prevents liar from getting t...,1
4,mother comes pretty close to using word 'strea...,1


In [121]:
data['is_sarcastic'].value_counts(normalize = True)

is_sarcastic
0    0.541679
1    0.458321
Name: proportion, dtype: float64

Бачимо, що значного дізбалансу в даних нема. Тому можемо не добавляти ваги для класів

In [122]:
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import re
batch_size = 32

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

class SarcasmDataset(Dataset):
    '''
    Класс сімейства Датасет для наших даних 
    '''
    def __init__(self, data, transform = None):
        '''
        Конструктор
        '''
        self.data = data
        self.transform = transform
        
        
    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, idx):

        item = self.data.iloc[idx]
        
        target = int(item['is_sarcastic'])
        sentence = item.headline
        'Токенайзуємо речення та екстрактимо токени зі масками'
        tokens = tokenizer.encode_plus(
            sentence,
            add_special_tokens = True,
            max_length=64,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        

        seq = tokens['input_ids']
        mask = tokens['attention_mask']
        
        if self.transform:
            seq = self.transform(seq)
        
        'Повертаємо токени зі масками речення, та класс, до якого належить оброблене речення'
        return {
            'input_ids': seq,
            'attention_mask': mask,
            'class': target
         }
    
dataset = SarcasmDataset(data) #Ініцюалізуємо
trainset, testset = random_split(dataset, [0.7, 0.3]) #Ділимо на train та test
#Та ініцюалізуємо train та test-дані 
train_loader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(testset, shuffle=True, batch_size=batch_size)

In [123]:
class BERT_Arch(nn.Module):

    def __init__(self):
        super(BERT_Arch, self).__init__()
        
        self.bert = AutoModel.from_pretrained('bert-base-uncased')

        
        self.fc1 = nn.Sequential(nn.Dropout(0.1),
                                nn.Linear(768,256),
                                nn.ReLU())
      
        self.fc2 = nn.Sequential(nn.Dropout(0.1),
                                nn.Linear(256,64),
                                nn.ReLU())

        self.fc3 = nn.Sequential(nn.Dropout(0.1),
                                nn.Linear(64,1))
    #define the forward pass
    def forward(self, sent_id, mask):
        
        #Берт Модель 
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
        #І поверх неї модель класифікації

        x = self.fc1(cls_hs)
        
        x = self.fc2(x)
        
        # output layer
        x = self.fc3(x)

        return x

In [124]:
model = BERT_Arch().to(dtype=torch.float16, device='cuda')


In [ ]:
from transformers import AdamW
import tqdm
from torchmetrics.classification import BinaryAccuracy
# define the optimizer
loss_f = torch.nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(),lr = 1e-5) 
epochs = 10
accuracy = BinaryAccuracy().to(device)
for i in range(epochs):
    print(f"Epoch {i}")
    # Train loop
    train_tqdm = tqdm.tqdm(train_loader)
    model.train()
    for batch in train_tqdm:
        input_ids, attention_mask, cls = batch['input_ids'], batch['attention_mask'], batch['class']
        
        # Передаємо до ГПУ наші тензори'
        input_ids, attention_mask, cls = input_ids.to(device), attention_mask.to(device), cls.to(device)
        #Онуляємо градієнти 
        optimizer.zero_grad()
        model.zero_grad()
        
        #Предиктимо класи
        output = model(input_ids.squeeze(1), attention_mask.squeeze(1)).squeeze(1)
        
        # Рахуємо трати
        L = loss_f(output, cls.float())
        
        acc = accuracy(softmax(output), cls)
        #Передаємо до моделі градієнт втрат
        L.backward()
        
        
        optimizer.step() #Оновлення параметрів отпимізатору
        train_tqdm.set_description(f"Train loss: {L.item()}  Accuracy -- {acc}") #Вивід у термінал значення втрат
        train_tqdm.refresh()


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0


  0%|          | 0/1211 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/tmp/ipykernel_35/4234795135.py:29: UserWarning: Implicit d

Epoch 1


Train loss: 0.6448150873184204  Accuracy -- 0.800000011920929: 100%|██████████| 1211/1211 [03:51<00:00,  5.24it/s]


Epoch 2


Train loss: 0.6726970672607422  Accuracy -- 0.5:   6%|▌         | 74/1211 [00:14<03:37,  5.24it/s]    